In [10]:
from bactrack.widget import get_hierarchies_from_masks_folder
mask_dir = "/home/tlo/Documents/Data/bactrack_test3/xy0/masks"
hier_arr = get_hierarchies_from_masks_folder(mask_dir)

In [11]:
from bactrack.tracking import OverlapWeight, IOUWeight, DistanceWeight
w =  OverlapWeight(hier_arr)

2024-04-12 13:31:57,216 [INFO] Weight function start computing weight matrix
2024-04-12 13:31:57,237 [INFO] Weight matrix computed, time consuming:0.019722700119018555 sec


In [12]:
#   MIPSolver => python_mip repo: it includes two solvers: CBC and Gurobi
#   ScipySolver => scipy.optimize.milp: it include HiGHS solvers

# MIPSolver require conda environment and additional setup, so it not always avilable, you could use 
# ScipySolver instead
from bactrack.tracking import MIPSolver, ScipySolver

# do solver = ScipySolver(w.weight_matrix, hier_arr, mask_penalty= w.mask_penalty)
# if you want include a mask_penalty calculated by omnipose/cellpose mask threshold + flow threshold
solver = MIPSolver(w.weight_matrix, hier_arr)

# n is a list of all selected segementation
# e is a scipy sparse matrix mark the edge be selected
n, e = solver.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12
2024-04-12 13:31:58,324 [INFO] Setting up MIP problem
2024-04-12 13:31:58,337 [INFO] MIP problem set up: time used 0.01183462142944336 sec
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 310 rows, 573 columns and 1146 nonzeros
Model fingerprint: 0x05a6271f
Variable types: 0 continuous, 573 integer (470 binary)
Coefficient statistics:
  Matrix range     [8e-03, 4e+09]
  Objective range  [2e-03, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting Numeri

In [13]:
from bactrack import io
# m is a mask array including all output masks
# e is a edge dataframe
# also selected cell in  hier_arr be marked in this function
m, e = io.format_output(hier_arr, n, e,overwrite_mask = False)
# now n is  a node dataframe
n = io.hiers_to_df(hier_arr)

In [14]:
# merge
import pandas as pd

merged_df = pd.merge(e, n.add_suffix('_source'), left_on='Source Index', right_on='index_source', how='left')
merged_df = pd.merge(merged_df, n.add_suffix('_target'), left_on='Target Index', right_on='index_target', how='left')

In [16]:
# export links to csv
import os

xydir = os.path.abspath(os.path.join(mask_dir, os.pardir)) #get xydir path
bactrackdir = os.path.join(xydir, "bactrackfiles")
os.makedirs(bactrackdir, exist_ok=True)  #make dir in xydir for bactrack file
merged_df.to_csv(os.path.join(bactrackdir, "mergedlinks.csv")) #save full merged links file 
# selected_df = merged_df[['frame_source','label_source', 'frame_target', 'label_target']]
selected_df = merged_df[['frame_source','label_source', 'label_target','area_source','area_target']]
selected_df.to_csv(os.path.join(bactrackdir, "superseggerlinks.csv")) #save links file relevant for supersegger